#  The Legend of Python: A Take-Home Final
Welcome, brave developer! Your journey begins now. May your code be clean and your bugs be few.

##  Quest 1: Retrieving the Scrolls of Time
The ancient Floatrates API holds the lost exchange rates. We must summon them using ancient XML incantations.
- We'll randomly choose a currency.
- Fetch data from May 4, 2011, to the present.
- Use `requests` and `xmltodict` to decode the scrolls.

In [ ]:
# Preparations and dependencies
import requests
import xmltodict
import random
from datetime import datetime, timedelta
import pandas as pd
import logging
import time

# Available currencies
currencies = [
    'EUR', 'GBP', 'USD', 'DZD', 'AUD', 'BWP', 'BND', 'CAD', 'CLP', 'CNY', 'COP',
    'CZK', 'DKK', 'HUF', 'ISK', 'INR', 'IDR', 'ILS', 'KZT', 'KRW', 'KWD', 'LYD',
    'MYR', 'MUR', 'NPR', 'NZD', 'NOK', 'OMR', 'PKR', 'PLN', 'QAR', 'RUB', 'SAR',
    'SGD', 'ZAR', 'LKR', 'SEK', 'CHF', 'THB', 'TTD'
]

# Set up logging
logging.basicConfig(filename='quest_log.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
# Retrieve historical data from the first date until today
base_currency = random.choice(currencies)
logging.info(f"Selected base currency: {base_currency}")
start_date = datetime(2011, 5, 4)
end_date = datetime.today()
delta = timedelta(days=30)  # interval to prevent server overload

all_data = []
current = start_date
while current <= end_date:
    date_str = current.strftime('%Y-%m-%d')
    url = (
        f"https://www.floatrates.com/historical-exchange-rates.html"
        f"?operation=rates&pb_id=1775&page=historical"
        f"&currency_date={date_str}&base_currency_code={base_currency}&format_type=xml"
    )
    try:
        response = requests.get(url)
        data = xmltodict.parse(response.content)
        all_data.append(data)
        logging.info(f"Successfully retrieved data for {date_str}")
    except Exception as e:
        logging.error(f"Failed on {date_str}: {str(e)}")
    current += delta
    time.sleep(0.2)  # Be gentle with the server


## Quest 2: Exploratory Data Analysis
Now that the scrolls have been retrieved, it’s time to divine their meaning. Let us inspect the fluctuations and patterns.

In [ ]:
# For the sake of demonstration, parse and normalize a sample of the XML into a DataFrame
exchange_records = []
for entry in all_data:
    try:
        items = entry['data']['list']['item']
        for i in items:
            exchange_records.append({
                'date': i['currencyDate'],
                'target': i['targetCurrencyCode'],
                'rate': float(i['exchangeRate'])
            })
    except Exception as e:
        logging.warning(f"Skipping malformed entry: {str(e)}")

df = pd.DataFrame(exchange_records)
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True)
df.head()

In [ ]:
# Plot a sample trend
import matplotlib.pyplot as plt
sample_currency = df['target'].value_counts().idxmax()
subset = df[df['target'] == sample_currency]

plt.figure(figsize=(10, 5))
plt.plot(subset['date'], subset['rate'])
plt.title(f'Exchange Rate Trend: {base_currency} to {sample_currency}')
plt.xlabel('Date')
plt.ylabel('Rate')
plt.grid(True)
plt.tight_layout()
plt.show()

##  Side Quest 1: Logging System
A robust logging system has been constructed throughout the notebook. Check `quest_log.log` for a full adventure log.

##  Side Quest 2: Code Profiling
Now, we assess the efficiency of our magic.

In [ ]:
import cProfile
cProfile.run("requests.get('https://www.floatrates.com')")

##  Quest Complete!
You have completed your journey. The economy of Pyrovia thanks you.